## FieldList from numpy array and metadata

In [1]:
import earthkit.data
from earthkit.data.core.fieldlist import FieldList
import numpy as np

earthkit.data.download_example_file("test6.grib")
ds = earthkit.data.from_source("file", "test6.grib")

In [2]:
ds[0].values

array([272.56417847, 272.56417847, 272.56417847, 272.56417847,
       272.56417847, 272.56417847, 272.56417847, 272.56417847,
       272.56417847, 272.56417847, 272.56417847, 272.56417847,
       288.56417847, 296.56417847, 288.56417847, 288.56417847,
       288.56417847, 288.56417847, 280.56417847, 288.56417847,
       296.56417847, 288.56417847, 280.56417847, 280.56417847,
       312.56417847, 304.56417847, 320.56417847, 296.56417847,
       304.56417847, 296.56417847, 296.56417847, 296.56417847,
       288.56417847, 296.56417847, 296.56417847, 296.56417847,
       296.56417847, 304.56417847, 296.56417847, 304.56417847,
       296.56417847, 296.56417847, 296.56417847, 296.56417847,
       296.56417847, 296.56417847, 296.56417847, 296.56417847,
       288.56417847, 296.56417847, 288.56417847, 288.56417847,
       280.56417847, 288.56417847, 288.56417847, 288.56417847,
       288.56417847, 288.56417847, 280.56417847, 288.56417847,
       272.56417847, 264.56417847, 264.56417847, 272.56

In [3]:
v = ds[0].values
v = v * 0 + 1
v = np.array([v])
v.shape

(1, 84)

In [4]:
r = FieldList.from_numpy(v, ds[0].metadata())

In [5]:
r.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll


In [6]:
r.values

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]])